In [1]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Flatten, AveragePooling2D, Dropout
from keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from os import getcwd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau

In [2]:
train_dir = 'C:/Users/Asus/COVID19 datasets - Limited (3 classes)/Training'
validation_dir = 'C:/Users/Asus/COVID19 datasets - Limited (3 classes)/Testing'

train_pneu_dir = train_dir+'/PNEUMONIA'+'/train/'
train_covid_dir = train_dir+'/COVID19'+'/train/'
train_normal_dir = train_dir+'/NORMAL'+'/train/'

validation_pneu_dir = validation_dir+'/PNEUMONIA'+'/test/'
validation_covid_dir = validation_dir+'/COVID19'+'/test/'
validation_normal_dir = validation_dir+'/NORMAL'+'/test/'

In [3]:
class myCallback(Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy')>=0.96):
            print("\nReached limit accuracy so cancelling training!")
            self.model.stop_training = True

In [4]:
ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=5,
    verbose=0,
    mode="auto",
    min_delta=0.0001,
    cooldown=0,
    min_lr=0
)

In [5]:
batch = 12

train_datagen = ImageDataGenerator(rescale = 1./255)

test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                   batch_size = batch,
                                                   class_mode = 'categorical',
                                                   target_size = (224, 224),
                                                   shuffle = True)     

validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                        batch_size = batch,
                                                        class_mode = 'categorical',
                                                        target_size = (224, 224),
                                                        shuffle = True)

Found 750 images belonging to 3 classes.
Found 150 images belonging to 3 classes.


In [6]:
local_weights_file = f"{getcwd()}/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5"

pre_trained_model = MobileNetV2(include_top = False, 
                               weights = None,
                               alpha = 1,
                               input_shape = (224, 224, 3))

pre_trained_model.load_weights(local_weights_file)
  
pre_trained_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                              

 block_3_expand_relu (ReLU)     (None, 56, 56, 144)  0           ['block_3_expand_BN[0][0]']      
                                                                                                  
 block_3_pad (ZeroPadding2D)    (None, 57, 57, 144)  0           ['block_3_expand_relu[0][0]']    
                                                                                                  
 block_3_depthwise (DepthwiseCo  (None, 28, 28, 144)  1296       ['block_3_pad[0][0]']            
 nv2D)                                                                                            
                                                                                                  
 block_3_depthwise_BN (BatchNor  (None, 28, 28, 144)  576        ['block_3_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_3_d

 lization)                                                                                        
                                                                                                  
 block_7_expand (Conv2D)        (None, 14, 14, 384)  24576       ['block_6_project_BN[0][0]']     
                                                                                                  
 block_7_expand_BN (BatchNormal  (None, 14, 14, 384)  1536       ['block_7_expand[0][0]']         
 ization)                                                                                         
                                                                                                  
 block_7_expand_relu (ReLU)     (None, 14, 14, 384)  0           ['block_7_expand_BN[0][0]']      
                                                                                                  
 block_7_depthwise (DepthwiseCo  (None, 14, 14, 384)  3456       ['block_7_expand_relu[0][0]']    
 nv2D)    

                                                                                                  
 block_10_depthwise_relu (ReLU)  (None, 14, 14, 384)  0          ['block_10_depthwise_BN[0][0]']  
                                                                                                  
 block_10_project (Conv2D)      (None, 14, 14, 96)   36864       ['block_10_depthwise_relu[0][0]']
                                                                                                  
 block_10_project_BN (BatchNorm  (None, 14, 14, 96)  384         ['block_10_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_11_expand (Conv2D)       (None, 14, 14, 576)  55296       ['block_10_project_BN[0][0]']    
                                                                                                  
 block_11_

 block_14_depthwise (DepthwiseC  (None, 7, 7, 960)   8640        ['block_14_expand_relu[0][0]']   
 onv2D)                                                                                           
                                                                                                  
 block_14_depthwise_BN (BatchNo  (None, 7, 7, 960)   3840        ['block_14_depthwise[0][0]']     
 rmalization)                                                                                     
                                                                                                  
 block_14_depthwise_relu (ReLU)  (None, 7, 7, 960)   0           ['block_14_depthwise_BN[0][0]']  
                                                                                                  
 block_14_project (Conv2D)      (None, 7, 7, 160)    153600      ['block_14_depthwise_relu[0][0]']
                                                                                                  
 block_14_

In [7]:
for layer in pre_trained_model.layers:
    layer.trainable = False

In [8]:
last_layer = pre_trained_model.get_layer('block_14_add')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 7, 7, 160)


In [9]:
x = AveragePooling2D((4, 4)) (last_output)
x = Flatten()(x)
x = Dense(64, activation = 'relu')(x)
x = Dropout(0.5)(x)                  
x = Dense(3, activation = 'softmax')(x)           

model = Model(pre_trained_model.input, x) 

model.compile(optimizer = Adam(learning_rate = 0.001), 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

 block_3_expand_relu (ReLU)     (None, 56, 56, 144)  0           ['block_3_expand_BN[0][0]']      
                                                                                                  
 block_3_pad (ZeroPadding2D)    (None, 57, 57, 144)  0           ['block_3_expand_relu[0][0]']    
                                                                                                  
 block_3_depthwise (DepthwiseCo  (None, 28, 28, 144)  1296       ['block_3_pad[0][0]']            
 nv2D)                                                                                            
                                                                                                  
 block_3_depthwise_BN (BatchNor  (None, 28, 28, 144)  576        ['block_3_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_3_d

 lization)                                                                                        
                                                                                                  
 block_7_expand (Conv2D)        (None, 14, 14, 384)  24576       ['block_6_project_BN[0][0]']     
                                                                                                  
 block_7_expand_BN (BatchNormal  (None, 14, 14, 384)  1536       ['block_7_expand[0][0]']         
 ization)                                                                                         
                                                                                                  
 block_7_expand_relu (ReLU)     (None, 14, 14, 384)  0           ['block_7_expand_BN[0][0]']      
                                                                                                  
 block_7_depthwise (DepthwiseCo  (None, 14, 14, 384)  3456       ['block_7_expand_relu[0][0]']    
 nv2D)    

                                                                                                  
 block_10_depthwise_relu (ReLU)  (None, 14, 14, 384)  0          ['block_10_depthwise_BN[0][0]']  
                                                                                                  
 block_10_project (Conv2D)      (None, 14, 14, 96)   36864       ['block_10_depthwise_relu[0][0]']
                                                                                                  
 block_10_project_BN (BatchNorm  (None, 14, 14, 96)  384         ['block_10_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_11_expand (Conv2D)       (None, 14, 14, 576)  55296       ['block_10_project_BN[0][0]']    
                                                                                                  
 block_11_

 block_14_depthwise (DepthwiseC  (None, 7, 7, 960)   8640        ['block_14_expand_relu[0][0]']   
 onv2D)                                                                                           
                                                                                                  
 block_14_depthwise_BN (BatchNo  (None, 7, 7, 960)   3840        ['block_14_depthwise[0][0]']     
 rmalization)                                                                                     
                                                                                                  
 block_14_depthwise_relu (ReLU)  (None, 7, 7, 960)   0           ['block_14_depthwise_BN[0][0]']  
                                                                                                  
 block_14_project (Conv2D)      (None, 7, 7, 160)    153600      ['block_14_depthwise_relu[0][0]']
                                                                                                  
 block_14_

In [10]:
callbacks = myCallback()
reduce_lr = ReduceLROnPlateau()
history = model.fit(train_generator,
                    validation_data = validation_generator,
                    verbose = 1,
                    epochs = 40,
                    batch_size = batch,
                    callbacks = [callbacks, reduce_lr])

Epoch 1/40
63/63 [==============================] - 115s 2s/step - loss: 0.9395 - accuracy: 0.6467 - val_loss: 0.2689 - val_accuracy: 0.9333 - lr: 0.0010
Epoch 2/40
63/63 [==============================] - 92s 1s/step - loss: 0.3702 - accuracy: 0.8680 - val_loss: 0.1860 - val_accuracy: 0.9400 - lr: 0.0010
Epoch 3/40
63/63 [==============================] - 139s 2s/step - loss: 0.2626 - accuracy: 0.9027 - val_loss: 0.1552 - val_accuracy: 0.9400 - lr: 0.0010
Epoch 4/40
63/63 [==============================] - 167s 3s/step - loss: 0.2396 - accuracy: 0.9120 - val_loss: 0.1314 - val_accuracy: 0.9533 - lr: 0.0010
Epoch 5/40
63/63 [==============================] - 194s 3s/step - loss: 0.2292 - accuracy: 0.9200 - val_loss: 0.1173 - val_accuracy: 0.9533 - lr: 0.0010
Epoch 6/40
63/63 [==============================] - 205s 3s/step - loss: 0.1851 - accuracy: 0.9267 - val_loss: 0.1276 - val_accuracy: 0.9600 - lr: 0.0010
Epoch 7/40
63/63 [==============================] - 215s 3s/step - loss: 0.15

In [11]:
results = model.evaluate(validation_generator, batch_size=128)
print("test loss, test acc:", results)

13/13 [==============================] - 17s 1s/step - loss: 0.0864 - accuracy: 0.9667
test loss, test acc: [0.08638253062963486, 0.9666666388511658]
